<a href="https://colab.research.google.com/github/fred-ykv/Value-Investing-In-Python/blob/master/Novo_Codigo_2025_FINVIZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------ #
# Importações
import pandas as pd
import numpy as np
import yfinance as yf
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ------------------ #
# Função para baixar HTML do Finviz
def get_finviz_html(symbol):
    print(f"Baixando HTML do Finviz para {symbol}...")

    url = f"https://finviz.com/quote.ashx?t={symbol.lower()}"
    print(f"URL gerada: {url}")

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

    try:
        req = Request(url, headers=headers)
        response = urlopen(req)
        webpage = response.read()
        print("HTML baixado com sucesso!")

        html = soup(webpage, "html.parser")
        return html

    except Exception as e:
        print(f"Erro ao baixar HTML: {e}")
        return None

# ------------------ #
# Função para extrair dados fundamentais (abordagem robusta)
def parse_finviz_data(html):
    print("Extraindo dados fundamentais do Finviz (abordagem robusta)...")

    atributos_fundamentais = ['P/E', 'Market Cap', 'ROE', 'ROI', 'EPS (ttm)', 'Debt/Eq', 'Dividend', 'Quick Ratio', 'Current Ratio']
    tables = html.find_all("table")
    data = {}

    for table in tables:
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            if len(cells) == 2:
                atributo = cells[0].get_text(strip=True)
                valor = cells[1].get_text(strip=True)
                if atributo in atributos_fundamentais:
                    data[atributo] = valor

    if data:
        print("✅ Dados fundamentais extraídos com sucesso!")
        return data
    else:
        print("⚠️ Tabela de fundamentos não encontrada ou sem dados relevantes.")
        return None

# ------------------ #
# Função para fallback (índice fixo)
def parse_finviz_data_by_index(html, index=9):
    print(f"Tentando extrair dados fundamentais usando índice fixo {index}...")
    tables = html.find_all("table")

    if len(tables) > index:
        table = tables[index]
        rows = table.find_all("tr")

        attributes = []
        values = []

        for row in rows:
            data_cells = [cell.text.strip() for cell in row.find_all("td")]
            if len(data_cells) > 0:
                for i, cell_text in enumerate(data_cells):
                    if i % 2 == 0:
                        attributes.append(cell_text)
                    else:
                        values.append(cell_text)

        df = pd.DataFrame({'Atributos': attributes, 'Valores': values})
        print("✅ Dados fundamentais extraídos via índice fixo!")
        return df
    else:
        print(f"⚠️ Índice {index} não encontrado. Número total de tabelas: {len(tables)}.")
        return None

# ------------------ #
# Função para extrair dados de insiders
def get_insider_data(html):
    print("Extraindo dados de transações de insiders...")

    try:
        tables = pd.read_html(str(html), attrs={'class': 'body-table'})
        if tables:
            insider_df = tables[0]
            insider_df.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
            insider_df = insider_df.set_index('Date')
            print("✅ Dados de insiders extraídos com sucesso!")
            return insider_df
        else:
            print("⚠️ Nenhuma tabela de insiders encontrada.")
            return None
    except Exception as e:
        print(f"⚠️ Erro ao extrair dados de insiders: {e}")
        return None

# ------------------ #
# Função para extrair notícias
def get_news_data(html):
    print("Extraindo notícias recentes...")

    try:
        news_table = html.find('table', class_='fullview-news-outer')
        if news_table is None:
            print("⚠️ Nenhuma tabela de notícias encontrada.")
            return None

        rows = news_table.find_all('tr')
        news_data = []
        for row in rows:
            date_cell = row.find('td', class_='nn-date')
            headline_cell = row.find('a', class_='tab-link-news')
            if date_cell and headline_cell:
                date = date_cell.get_text(strip=True)
                headline = headline_cell.get_text(strip=True)
                link = 'https://finviz.com' + headline_cell['href']
                news_data.append({'Date': date, 'Headline': headline, 'Link': link})

        news_df = pd.DataFrame(news_data)
        print("✅ Notícias extraídas com sucesso!")
        return news_df
    except Exception as e:
        print(f"⚠️ Erro ao extrair notícias: {e}")
        return None

# ------------------ #
# Fluxo principal
def main():
    # Pede o ticker ao usuário
    symbol = input("Digite o ticker da ação: ").strip().upper()

    # Baixa HTML
    html = get_finviz_html(symbol)
    if html is None:
        print("⚠️ Falha ao obter o HTML. Encerrando o script.")
        return

    # Extrai dados fundamentais (abordagem robusta + fallback)
    finviz_data = parse_finviz_data(html)
    if finviz_data:
        print("\n✅ Dados Fundamentais (abordagem robusta):")
        for k, v in finviz_data.items():
            print(f"{k}: {v}")
    else:
        print("⚠️ Abordagem robusta falhou. Tentando abordagem por índice fixo...")
        df_finviz = parse_finviz_data_by_index(html)
        if df_finviz is not None:
            print("\n✅ Dados Fundamentais (fallback por índice fixo):")
            print(df_finviz)
        else:
            print("⚠️ Nenhuma tabela de fundamentos encontrada.")

    # Extrai dados de insiders
    insider_df = get_insider_data(html)
    if insider_df is not None:
        print("\n✅ Transações de Insiders:")
        print(insider_df)
    else:
        print("⚠️ Nenhuma transação de insider encontrada.")

    # Extrai notícias recentes
    news_df = get_news_data(html)
    if news_df is not None:
        print("\n✅ Notícias recentes:")
        print(news_df)
    else:
        print("⚠️ Nenhuma notícia encontrada.")

# ------------------ #
# Executa
if __name__ == "__main__":
    main()

Digite o ticker da ação: mli
Baixando HTML do Finviz para MLI...
URL gerada: https://finviz.com/quote.ashx?t=mli
HTML baixado com sucesso!
Extraindo dados fundamentais do Finviz (abordagem robusta)...
⚠️ Tabela de fundamentos não encontrada ou sem dados relevantes.
⚠️ Abordagem robusta falhou. Tentando abordagem por índice fixo...
Tentando extrair dados fundamentais usando índice fixo 9...
✅ Dados fundamentais extraídos via índice fixo!

✅ Dados Fundamentais (fallback por índice fixo):
       Atributos    Valores
0          Index        RUT
1            P/E      14.11
2      EPS (ttm)       5.49
3    Insider Own      3.55%
4   Shs Outstand    110.65M
..           ...        ...
73         SMA50      2.95%
74        SMA200     -0.50%
75        Trades           
76        Volume  1,421,847
77        Change      0.03%

[78 rows x 2 columns]
Extraindo dados de transações de insiders...
✅ Dados de insiders extraídos com sucesso!

✅ Transações de Insiders:
                                Tra